In [2]:
import numpy as np
import torch
import pandas as pd
import joblib
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn

In [3]:
class ChurnModel(nn.Module):
  def __init__(self, input_dim):

    super(ChurnModel, self).__init__()

    self.layers = nn.Sequential(

        nn.Linear(input_dim, 256), # The first layer
        nn.BatchNorm1d(256),
        nn.ReLU(),
        nn.Dropout(0.2),

        nn.Linear(256, 128), # The second layer
        nn.BatchNorm1d(128),
        nn.ReLU(),
        nn.Dropout(0.2),

        nn.Linear(128, 64), # 3th layer
        nn.BatchNorm1d(64),
        nn.ReLU(),
        nn.Dropout(0.2),

        nn.Linear(64, 32), # 4th layer
        nn.ReLU(),

        nn.Linear(32,1), # output layer
        nn.Sigmoid()

    )

  # Define forward pass
  def forward(self,x):
    return self.layers(x)

In [ ]:
def predict_from_csv(csv_path, model_path, scaler_path):
    df = pd.read_csv(csv_path)

    # input features
    features = [
        'gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure', 
        'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 
        'DeviceProtection', 'TechSupport', 'StreamingMovies', 'Contract', 
        'PaperlessBilling', 'PaymentMethod', 'MonthlyCharges', 'TotalCharges'
    ]
    X_new = df[features]

    # Download StandardScaler
    scaler = joblib.load(scaler_path)
    X_new = scaler.transform(X_new.to_numpy())  

    # Convert to PyTorch tensor
    X_new_tensor = torch.tensor(X_new, dtype=torch.float32)

    # Load the model
    input_dim = X_new.shape[1] 
    model = ChurnModel(input_dim)
    model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
    model.eval()

    # Predictions
    with torch.no_grad():
        predictions = model(X_new_tensor)
        predicted_labels = (predictions > 0.5).float() 


    df['Churn Prediction'] = pd.Series(predicted_labels.numpy().astype(int).squeeze()).map({0: "Stay", 1: "Will Churn"})

    
    return df[['Churn Prediction']]

csv_file = r'F:\AI\VSC\Project1\src\df2.csv'
model_file = r'F:\AI\VSC\Project1\models\churn_model.pth'
scaler_file = r'F:\AI\VSC\Project1\models\scaler.pkl'

predictions = predict_from_csv(csv_file, model_file, scaler_file)
predictions

,Churn Prediction
0,Stay
1,Stay
2,Stay
3,Stay
4,Stay
5,Stay
6,Stay
7,Stay
8,Stay
9,Stay
